In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
import base64, sys, socket, time, os
import Crypto.Cipher.AES
import CBC,Scaler, RSA_python
import IP

tf.set_random_seed(777)  # reproducibility

print("[초기 구동 방식은 동일 디렉토리의 readme.txt 참조]")
print("접속하고자 하는 IP 주소와 포트번호를 입력해주세요 : ")
TCP_IP, TCP_PORT = input().split()
TCP_PORT = int(TCP_PORT)

# IP 유효성 확인
if IP.ip_check(TCP_IP) == False:
    exit()

BUFFER_SIZE = 2048

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((TCP_IP, TCP_PORT))
message = 'Connected'
sock.sendall(message.encode('utf-8'))

# train Parameters
filename = '(en)parameters.txt'
password = input("Please Input PASSWORD! : ")
CBC.decrypt(CBC.getKey(password), filename)
filename = '(de)' + filename[4:]
with open(filename, 'r') as f:
    r = f.read().split(',')

# 여기서 비밀번호가 틀렸는지 맞았는지를 알려주면 그것조차도 공격자에게 정보를 주게 되는 것이므로 알려주지 않음
filename = '(en)data.csv'
password = input("Please Input PASSWORD! : ")
CBC.decrypt(CBC.getKey(password), filename)
filename = '(de)' + filename[4:]
xy = np.loadtxt(filename, delimiter=',')
#print(xy)

x = xy[:, 0:int(r[1])]
nor_min = np.min(x, 0) 
nor_den = np.max(x, 0) - np.min(x, 0) 

x = Scaler.MinMaxScaler(x)
y = xy[:, [-1]]  

# build a dataset
dataX = []
dataY = []
cnt = 0

for i in range(0, len(y) ,int(r[0])): 
    _x = x[i:i + int(r[0])] 
    _y = y[i + int(r[0]) - 1]  # Present label
    
    dataX.append(_x)
    dataY.append(_y)

test_size = len(dataY) - int(r[7])
trainX, testX = np.array(dataX[0:int(r[7])]), np.array(
    dataX[int(r[7]):len(dataX)])
trainY, testY = np.array(dataY[0:int(r[7])]), np.array(
    dataY[int(r[7]):len(dataY)])

# input place holders
X = tf.placeholder(tf.float32, [None, int(r[0]), int(r[1])])
Y = tf.placeholder(tf.float32, [None, int(r[3])])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units = int(r[2]), state_is_tuple = True, activation = tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], int(r[3]), activation_fn = None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares

# optimizer
optimizer = tf.train.AdamOptimizer(float(r[4]))
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None,int(r[3])])
predictions = tf.placeholder(tf.float32, [None, int(r[3])])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

# SAVE
saver = tf.train.Saver()
    
# 학습 모델 읽어와 TEST 하는 부분
# Running a new session
print("Starting reading session...")
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess,r[6])
    message = "R\n"
    message = RSA_python.PyEncryption(message) 
    sock.sendall(message)

    while True:
        dataX_t = []
        input_str = sock.recv(BUFFER_SIZE)
        tmp = RSA_python.PyDecryption(input_str)
        start_time = time.time()
        s_data = tmp.split(','.encode('utf-8'))
        i_data = [float (i) for i in s_data]
        i_data = np.reshape(i_data, (int(r[0]), int(r[1])))
        
        _numerator = i_data - nor_min
        i_data =  _numerator / (nor_den + 1e-7)
        #print(i_data)
        dataX_t.append(i_data)
        
        input_t = np.array(dataX_t[0:1])        
        test_predict = sess.run(Y_pred, feed_dict={X: input_t})
        print(test_predict)
        
        message = "{}\n".format(test_predict[0][0])
        message = RSA_python.PyEncrpytion(message)
        sock.sendall(message.encode('utf-8'))
        end_time = time.time()
    
sock.close()

[초기 구동 방식은 동일 디렉토리의 readme.txt 참조]
접속하고자 하는 IP 주소와 포트번호를 입력해주세요 : 
